## Get Hazmat definition and examples
First, what is the definition of a Hazmat? This [Wiki page](https://en.wikipedia.org/wiki/Dangerous_goods) contains a complete definition, but is rather verbose. Given that this definition is very important for our use case here, my recommended approach is to generate a draft of the definition based on the wiki page, and then validate with specialist. I'll also use English for the prompts because it usually provides better answers.

In [ ]:
from defs_and_tools import call_llm
import requests
from docling.document_converter import DocumentConverter
from html_to_markdown import convert_to_markdown
from dotenv import load_dotenv

load_dotenv()

# model="groq/llama-3.3-70b-versatile"
model="gemini/gemini-2.5-flash"

system = """
Define a Hazmat (Hazardous Material) based on the article provided by the user. 
The definition will be used as a reference for classifying products as Hazmat or not, so it must be concise and clear, focusing on the key aspects of what constitutes a Hazmat.
Also, extract from the article as many examples as possible, with the reason why each example is considered a Hazmat.
"""

wiki_article = "Dangerous_goods"

c:\Users\Guilherme\Documents\Projects\hazmat-classifier\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
url = f"https://en.wikipedia.org/w/api.php?format=json&action=query&prop=extracts&exlimit=max&explaintext&titles={wiki_article}"

# Get the page
response = requests.get(url)
data = response.json()

# Extract HTML content
html_content = data

print(html_content['query']['pages']['1476975']['extract'])

Dangerous goods are substances that are a risk to health, safety, property or the environment during transport. Certain dangerous goods that pose risks even when not being transported are known as hazardous materials (syllabically abbreviated as HAZMAT or hazmat). An example of dangerous goods is hazardous waste which is waste that threatens public health or the environment.
Hazardous materials are often subject to chemical regulations. Hazmat teams are personnel specially trained to handle dangerous goods, which include materials that are radioactive, flammable, explosive, corrosive, oxidizing, asphyxiating, biohazardous, toxic, poisonous, pathogenic, or allergenic. Also included are physical conditions such as compressed gases and liquids or hot materials, including all goods containing such materials or chemicals, or may have other characteristics that render them hazardous in specific circumstances.
Dangerous goods are often indicated by diamond-shaped signage on the item (see NFPA

This API output removed a main table that is in the middle of the article. Getting the raw HTML and converting to Markdown using Docling removed it as well. I will now use the lib html_to_markdown to convert.

In [4]:
url = "https://en.wikipedia.org/w/api.php"

params = {
    'action': 'parse',
    'page': wiki_article,
    'format': 'json',
    'prop': 'text'
}

# Get the page
response = requests.get(url, params=params)
data = response.json()

# Extract HTML content
html_content = data['parse']['text']['*']

markdown = convert_to_markdown(html_content)
print(markdown)

Solids, liquids, or gases harmful to people, other organisms, property or the environment
"HazMat" redirects here. For other uses, see [Hazmat (disambiguation)](/wiki/Hazmat_(disambiguation) "Hazmat (disambiguation)").
"Dangerous cargo" redirects here. For the 1954 film, see [Dangerous Cargo](/wiki/Dangerous_Cargo "Dangerous Cargo").
[<img src='//upload.wikimedia.org/wikipedia/commons/thumb/b/b9/HAZMAT_training.jpg/250px-HAZMAT_training.jpg' alt='' title='' width='250' height='166' />](/wiki/File:HAZMAT_training.jpg)

An emergency medical technician team training as rescue (grey suits) and decontamination (green suits) respondents to hazardous material and toxic contamination situations


[<img src='//upload.wikimedia.org/wikipedia/commons/thumb/5/58/GHS-pictogram-skull.svg/250px-GHS-pictogram-skull.svg.png' alt='' title='' width='250' height='250' />](/wiki/File:GHS-pictogram-skull.svg)

The [pictogram](/wiki/GHS_hazard_pictograms "GHS hazard pictograms") for poisonous substances of t

Using html-to-markdown, the table was successfully translated. I will use this as input to the workflow agent that summarizes the definition.

In [5]:
hazmat_def = call_llm(
    system=system,
    prompt=markdown,
    model=model)

In [ ]:
# save hazmat definition to a file

with open("data/hazmat-definition.md", "w") as f:
    f.write(hazmat_def)
    

The generated Hazmat definition is a draft that should be submitted to a expert to validate the definition and the examples. For me, it seems detailed and complete, with good examples, so I'll use it as is.